In [ ]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

In [ ]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "compute-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

# can poll for a minimum number of nodes and for a specific timeout. 
# if no min node count is provided it uses the scale settings for the cluster
compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

In [ ]:

import os

project_folder = './capstone-project'
os.makedirs(project_folder, exist_ok=True)

In [ ]:
import shutil

shutil.copy('train.py', project_folder)

# Create an experiment

In [ ]:
from azureml.core import Experiment

experiment_name = 'House_Price_Predication'
experiment = Experiment(ws, name=experiment_name)

# Dataset

In [ ]:
# from azureml.core.authentication import ServicePrincipalAuthentication
# svc_pr_password = os.environ.get("AZUREML_PASSWORD")

# svc_pr = ServicePrincipalAuthentication(
#     tenant_id="",
#     service_principal_id="",
#     service_principal_password="")


# ws = Workspace(subscription_id="",
#                resource_group="azureML",
#                workspace_name="creditcard",
#                auth=svc_pr)

# print("Found workspace {} at location {}".format(ws.name, ws.location))


# # ws = Workspace.from_config()
# # ws.auth=svc_pr
# experiment_name = 'House_Price_Predication'
# experiment=Experiment(ws, experiment_name)

# datastore=ws.get_default_datastore()
# dataset=Dataset.Tabular.from_delimited_files(datastore.path('UI/02-09-2021_034445_UTC/BankChurners.csv'))


# os.makedirs('data',exist_ok=True)
# local_path='data/prepared_data.csv'
# workspace=Workspace(ws.subscription_id,ws.resource_group,ws._workspace_name)
# # x_df=dataset.to_pandas_dataframe().head()
# # cat_cols=[col for col in x_df if x_df[col].dtype =='O']
# # num_cols=[col for col in x_df if x_df[col].dtype !='O']
# # pd.get_dummies(x_df).columns

## Create an environment
Define a conda environment YAML file with your training script dependencies and create an Azure ML environment.

In [ ]:
%%writefile conda_dependencies.yml

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-train-automl-runtime==1.21.0
  - inference-schema
  - azureml-interpret==1.21.0
  - azureml-defaults==1.21.0
- numpy>=1.16.0,<1.19.0
- pandas==0.25.1
- scikit-learn==0.22.1
- xgboost<=1.3.3
- psutil>=5.2.2,<6.0.0
channels:
- anaconda
- conda-forge

In [ ]:
from azureml.core import Environment

env = Environment.from_conda_specification(name = 'capstone-project-env', file_path = './conda_dependencies.yml')

In [ ]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory=project_folder,
                      script='train.py',
                      arguments=['--max_depth', '5'
                                 ,'--learning_rate',  0.1
                                 ,'--colsample_bytree',0.3
                                 ,'--alpha',10
                                 ,'--n_estimators',10],
                      compute_target=compute_target,
                      environment=env)

In [ ]:
run = experiment.submit(src)

# Monitor The Run

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()
#run.wait_for_completion(show_output=True)

# Tune model hyperparameters

In [ ]:
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import choice,uniform, randint
from azureml.train.hyperdrive.policy import BanditPolicy, MedianStoppingPolicy
from azureml.train.hyperdrive.parameter_expressions import uniform, randint, choice
from sklearn.metrics import mean_squared_error,accuracy_score

early_termination_policy = BanditPolicy(slack_factor=0.01)
another_early_termination_policy = MedianStoppingPolicy(evaluation_interval=1, delay_evaluation=5)

## Hyper Parameter Optimization
hyperparameter_grid = RandomParameterSampling({
    '--max_depth':choice(2, 3, 5, 10),
    '--learning_rate':choice(0.05,0.1,0.15,0.20),
    '--colsample_bytree':choice(0.3,0.5,0.7,0.9),
    '--alpha':choice(10,20,30,40),
    '--n_estimators':choice(100, 500, 900, 1100)
    }
)
                
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=hyperparameter_grid, 
                                     primary_metric_name='mean_squared_error',
                                     primary_metric_goal=PrimaryMetricGoal.MINIMIZE,
                                     policy=early_termination_policy,
                                     max_total_runs=10,
                                     max_concurrent_runs=5)

In [ ]:
# start the HyperDrive run
hyperdrive_run = experiment.submit(hyperdrive_config)

# Monitor HyperDrive runs

In [ ]:
RunDetails(hyperdrive_run).show()

In [ ]:
hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
assert(hyperdrive_run.get_status() == "Completed")

In [ ]:
hyperdrive_run.id

# Find and register best model

In [ ]:
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']
best_run

# Retrieve the Best Model

In [ ]:
import joblib
best_hyperdrive_model = best_run.register_model(
    model_name="House_Price_Model",
    model_path='outputs/house_price_model.pkl',
   
)
best_run.download_file("outputs/house_price_model.pkl", "outputs/house_price_model.pkl")

# Model Deployment

In [ ]:
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
#from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import Model

In [ ]:
aci_config = AciWebservice.deploy_configuration(
            cpu_cores=1,
            memory_gb=4, 
            enable_app_insights=True,
            auth_enabled=True,
            tags={"data":"house price regression"},
            description='house price regression Model',
            )

In [ ]:
inference_config = InferenceConfig(entry_script='score.py', environment=env)
inference_config

In [ ]:
service = Model.deploy(workspace=ws,
                       name="house-price-ml-service",
                       models=[best_hyperdrive_model], 
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)

service.wait_for_deployment(show_output=True)
print(service.state)

In [ ]:
logs = service.get_logs()
for line in logs.split('\n'):
     print(line)

In [51]:
# print service state
print(service.state)
# print scoring URI
print('scoring URI: ' + service.scoring_uri)
# print Swagger URI
print('Swagger URI: ' + service.swagger_uri)
# retrieve authentication keys
primary, secondary = service.get_keys()
# print primary authenticaton key
print('Primary Authentication Key: ' + primary)

Healthy
scoring URI: http://b7e34bc0-f1f3-4b47-8837-c46f3e8899af.southcentralus.azurecontainer.io/score
Swagger URI: http://b7e34bc0-f1f3-4b47-8837-c46f3e8899af.southcentralus.azurecontainer.io/swagger.json
Primary Authentication Key: SEMOR6g83ld8yjaB5OV05dxuct8eillI


In [ ]:
#Store the uri's in variables:
scoring_uri = 'http://b7e34bc0-f1f3-4b47-8837-c46f3e8899af.southcentralus.azurecontainer.io/score'

key = 'SEMOR6g83ld8yjaB5OV05dxuct8eillI'

# Consume the Endpoint and Testing

In [84]:
# #connect to dataset
# #https://medium.com/analytics-vidhya/deploy-your-ml-models-using-5-easy-steps-with-azure-machine-learning-workspace-c1ca5b6aa284
# dataset = Dataset.get_by_name(ws, name='<Name of dataset in AMLW>')
# dataset = dataset.to_pandas_dataframe()

# #package and run input data to model
# #input data
# input_data = dataset.to_json()

# #run model
# pred = service.run(input_data)
# #Convert returned json back to a pandas dataframe
# pred = pd.read_json(pred)

from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Workspace, Dataset

dataset_test = Dataset.Tabular.from_delimited_files(path='https://raw.githubusercontent.com/ddgope/Udacity-Capstone-House-Price-Predication-Using-Azure-ML/master/testdata.csv')
df_Test = dataset_test.to_pandas_dataframe()
df_Test.drop(['Column1','SalePrice'],axis=1,inplace=True) 
#df_Test.head()
#df_Test.shape

In [85]:
#input data
input_data = df_Test.to_json(orient="table",index=False)
#print(input_data)
input_data=json.dumps({"data": json.loads(input_data)["data"]},indent=4)
#run model
pred = service.run(input_data)
#Convert returned json back to a pandas dataframe

In [86]:
#print the result
pred

[126638.40625,
 142337.09375,
 181725.984375,
 191959.046875,
 196545.09375,
 180800.21875,
 181967.8125,
 173362.0,
 183880.28125,
 121203.6640625,
 203630.9375,
 89822.6171875,
 91773.4453125,
 148395.34375,
 114445.5625,
 389432.78125,
 257567.875,
 303245.5,
 278602.8125,
 407535.75,
 375848.6875,
 200041.109375,
 167621.546875,
 175675.03125,
 174541.75,
 201660.109375,
 361220.21875,
 231315.59375,
 220755.1875,
 202141.375,
 181407.375,
 94866.6875,
 185226.28125,
 313502.0625,
 278773.53125,
 238017.34375,
 168748.0,
 153833.328125,
 154441.671875,
 162729.640625,
 169366.078125,
 147028.1875,
 315835.0,
 222973.953125,
 229090.109375,
 194039.640625,
 258663.125,
 199021.265625,
 150645.140625,
 145923.796875,
 126261.59375,
 161116.890625,
 142399.515625,
 145019.203125,
 182636.828125,
 168597.828125,
 142082.296875,
 135622.1875,
 216582.5625,
 123833.4375,
 135064.40625,
 161150.78125,
 105301.7109375,
 123780.7578125,
 129537.0390625,
 132971.921875,
 105033.8984375,
 132

In [66]:
#df_Test.to_json('./testdata.json')

In [95]:
#let's test requests:
import json
import requests

scoring_uri = scoring_uri
key = key

headers = {'Content-Type':'application/json'}
headers['Authorization'] = f'Bearer {key}'

# Convert to JSON string
input_data = df_Test.to_json(orient="table",index=False)
#print(input_data)
input_data=json.dumps({"data": json.loads(input_data)["data"]},indent=4)

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

#load the returned prediction and read it into a pandas dataframe
pred = json.loads(resp.text)
pred = pd.read_json(pred)

[126638.40625, 142337.09375, 181725.984375, 191959.046875, 196545.09375, 180800.21875, 181967.8125, 173362.0, 183880.28125, 121203.6640625, 203630.9375, 89822.6171875, 91773.4453125, 148395.34375, 114445.5625, 389432.78125, 257567.875, 303245.5, 278602.8125, 407535.75, 375848.6875, 200041.109375, 167621.546875, 175675.03125, 174541.75, 201660.109375, 361220.21875, 231315.59375, 220755.1875, 202141.375, 181407.375, 94866.6875, 185226.28125, 313502.0625, 278773.53125, 238017.34375, 168748.0, 153833.328125, 154441.671875, 162729.640625, 169366.078125, 147028.1875, 315835.0, 222973.953125, 229090.109375, 194039.640625, 258663.125, 199021.265625, 150645.140625, 145923.796875, 126261.59375, 161116.890625, 142399.515625, 145019.203125, 182636.828125, 168597.828125, 142082.296875, 135622.1875, 216582.5625, 123833.4375, 135064.40625, 161150.78125, 105301.7109375, 123780.7578125, 129537.0390625, 132971.921875, 105033.8984375, 132063.828125, 142257.671875, 168787.25, 117174.953125, 99297.90625, 1

ValueError: Invalid file path or buffer object type: <class 'list'>

In [96]:
type(pred)

list

# Create the Submission File

In [97]:
##Create Sample Submission file and Submit using ANN
pred=pd.DataFrame(pred)
sub_df=pd.read_csv('https://raw.githubusercontent.com/ddgope/Udacity-Capstone-House-Price-Predication-Using-Azure-ML/master/sample_submission.csv')
datasets=pd.concat([sub_df['Id'],pred],axis=1)
datasets.columns=['Id','SalePrice']
datasets.to_csv('sample_submission.csv',index=False)